# Домашняя работа

В этой работы мы будем учиться предсказывать зарплату data scientist-ов в зависимочти от ряда факторов с помощью градиентоного бустинга

Цель домашней работы научиться предсказывать зарплаты (salary_in_usd) по ряды факторов

work_year: The number of years of work experience in the field of data science.

experience_level: The level of experience, such as Junior, Senior, or Lead.

employment_type: The type of employment, such as Full-time or Contract.

job_title: The specific job title or role, such as Data Analyst or Data Scientist.

salary: The salary amount for the given job.

salary_currency: The currency in which the salary is denoted.

salary_in_usd: The equivalent salary amount converted to US dollars (USD) for comparison purposes.

employee_residence: The country or region where the employee resides.

remote_ratio: The percentage of remote work offered in the job.

company_location: The location of the company or organization.

company_size: The company’s size is categorized as Small, Medium, or Large.

In [77]:
import pandas as pd

In [78]:
df=pd.read_csv("https://raw.githubusercontent.com/hse-ds/iad-intro-ds/master/2023/homeworks/hw-08-boosting/ds_salaries.csv")
#df = pd.read_csv("ds_salaries.csv") скачаю ссылкой, чтоб не грузить файл каждый раз

In [79]:
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


## Задание 1 (0.5 балла)

Разделите выборку на train, val, test (80%, 10%, 10%) в качестве таргета выберите salary_in_usd, удалите из признаков (salary) чтобы избежать лика в данных.

In [80]:
df.drop('salary',axis=1, inplace=True)#убрали salary
def num_year(x):
  return 2024-x #Поскольку у нас в описании признаков количество лет работы, то преобразуем год в количество лет, будем считать, что 2023 - это 1 год (поэтому вычитаем из 2024)
df['work_year']=df['work_year'].map(num_year)
df.head() #посмотрим, что получилось

,work_year,experience_level,employment_type,job_title,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,1,SE,FT,Principal Data Scientist,EUR,85847,ES,100,ES,L
1,1,MI,CT,ML Engineer,USD,30000,US,100,US,S
2,1,MI,CT,ML Engineer,USD,25500,US,100,US,S
3,1,SE,FT,Data Scientist,USD,175000,CA,100,CA,M
4,1,SE,FT,Data Scientist,USD,120000,CA,100,CA,M


In [81]:
from sklearn.model_selection import train_test_split
train_big, test=train_test_split(df, test_size=0.1, random_state=42) #разделили на тестовую и тренировочную (в тренировочную включили валидационную)
val_dolya=0.1/0.9 #Определили долю валидационной выборки, учитывая то, что она включена в общую тренировочную
train, val=train_test_split(train_big, test_size=val_dolya, random_state=42)#Разделили на валидационную и тренировочную
X_train, y_train=train.drop('salary_in_usd', axis=1), train['salary_in_usd'] #Разделили на признаки и целевую переменную для каждой выборки
X_val, y_val=val.drop('salary_in_usd', axis=1), val['salary_in_usd']
X_test, y_test=test.drop('salary_in_usd', axis=1), test['salary_in_usd']

## Задание 2 (0.5 балла)

Обучите модель линейной регресии и оцените её качество через mape и rmse

In [82]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error
X_train_new=X_train.select_dtypes(exclude=['object']) #Убираем все категориальные признаки и обучаем без них
X_val_new=X_val.select_dtypes(exclude=['object'])
X_test_new=X_test.select_dtypes(exclude=['object'])
model=LinearRegression()
model.fit(X_train_new, y_train) #Обучили на тренировочной
y_pred_val=model.predict(X_val_new) #Предсказали
mape_val=mean_absolute_percentage_error(y_val, y_pred_val) #Нашли mape
rmse_val=mean_squared_error(y_val, y_pred_val)**0.5 #Нашли rmse
print('MAPE на валидационной выборке:', mape_val)
print('RMSE на валидационной выборке:', rmse_val)
y_pred_test=model.predict(X_test_new) #То же самое, но для тестовой
mape_test=mean_absolute_percentage_error(y_test, y_pred_test)
rmse_test=mean_squared_error(y_test, y_pred_test)**0.5
print('MAPE на тестовой выборке:', mape_test)
print('RMSE на тестовой выборке:', rmse_test)

MAPE на валидационной выборке: 0.596725125157926
RMSE на валидационной выборке: 63923.02821778568
MAPE на тестовой выборке: 0.780099137585095
RMSE на тестовой выборке: 62081.08956935913


## Задание 3 (0.5 балла)

Обучите модель бустинга выберите любую из трех библиотек catboost, xgboost, lightgbm и оцените её качество через mape и rmse

In [83]:
!pip install --upgrade catboost
import catboost
from catboost import CatBoostRegressor #Модель, выбрала catboost
model=CatBoostRegressor()
model.fit(X_train_new, y_train) #Обучили на тренировочной
y_pred_val=model.predict(X_val_new) #Предсказали
mape_val=mean_absolute_percentage_error(y_val, y_pred_val) #Нашли mape
rmse_val=mean_squared_error(y_val, y_pred_val)**0.5 #Нашли rmse
print('MAPE на валидационной выборке:', mape_val)
print('RMSE на валидационной выборке:', rmse_val)
y_pred_test=model.predict(X_test_new) #То же самое, но для тестовой
mape_test=mean_absolute_percentage_error(y_test, y_pred_test)
rmse_test=mean_squared_error(y_test, y_pred_test)**0.5
print('MAPE на тестовой выборке:', mape_test)
print('RMSE на тестовой выборке:', rmse_test)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Learning rate set to 0.048712
0:	learn: 62288.2086020	total: 621us	remaining: 621ms
1:	learn: 62086.0107500	total: 1.46ms	remaining: 729ms
2:	learn: 61877.7315495	total: 2.32ms	remaining: 770ms
3:	learn: 61755.1790710	total: 3.19ms	remaining: 793ms
4:	learn: 61596.0977719	total: 4.44ms	remaining: 884ms
5:	learn: 61431.1974220	total: 5.55ms	remaining: 920ms
6:	learn: 61280.9351530	total: 6.12ms	remaining: 868ms
7:	learn: 61160.5900973	total: 7.41ms	remaining: 919ms
8:	learn: 61034.3692539	total: 8.08ms	remaining: 890ms
9:	learn: 60962.2743474	total: 8.8ms	remaining: 872ms
10:	learn: 60853.8941037	total: 9.66ms	remaining: 869ms
11:	learn: 60755.2089688	total: 10.6ms	remaining: 874ms
12:	learn: 60701.2190024	total: 11.3ms	remaining: 861ms
13:	learn: 60627.0162883	total: 12.1ms	remaining: 853ms
14:	learn: 60581.5455070	total: 12.8ms	remaining: 839ms
15:	learn: 60507.5996468	total: 13.8ms	rema

## Задание 4 (0.5 балла)

Далее предобработайте категориальные признаки с помощью onehot кодирования, labelencoder и счетчиков (mean target encoding) сравните качество моделей с помощью mape и rmse. Необходимо обучить две модели линейную и бустинг. Какие выводы можно сделать? Выберите лучшую из моделей и дальше будем работать с ней

In [84]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
!pip install category_encoders
from category_encoders import TargetEncoder
# OneHot- категориальные, но нет порядка: employment type (нельзя сравнить фулл и контракт), job title (нельзя сравнить аналитика и scientista) - при реализации оказалось, что здесь гораздо больше 2х значений, но я оставлю one hot
# labelencoder - порядок есть: experience (чем больше опыта - тем круче), company size (чем больше компания - тем лучше)
# счетчики - нет порядка, но очень много уникальных значений: salary currency, employee residence,company location - отнесла сюда признаки, в описании которых не были указаны значения
one_hot_col=['employment_type','job_title']
lab_col=['experience_level','company_size']
chet_col=['salary_currency','employee_residence','company_location'] #Названия столбцов, которые будем кодировать
ohe=OneHotEncoder(sparse=False)
le=LabelEncoder()
te=TargetEncoder()
df_target=df['salary_in_usd']
df_prizn=df
df_prizn.drop('salary_in_usd',axis=1, inplace=True)
ohe_df=pd.DataFrame(ohe.fit_transform(df_prizn[one_hot_col]), columns=ohe.get_feature_names_out(one_hot_col)) #One-hot
df_prizn=df_prizn.drop(one_hot_col, axis=1) #Убрали из начального дф эти столбцы
df_prizn=df_prizn.reset_index(drop=True) #сбросили индексы
df_prizn=pd.concat([df_prizn, ohe_df], axis=1) #Соединили
df_prizn[lab_col[0]]=le.fit_transform(df_prizn[lab_col[0]]) #LabelEncoder
df_prizn[lab_col[1]]=le.fit_transform(df_prizn[lab_col[1]])
df_prizn[chet_col] = te.fit_transform(df_prizn[chet_col], df_target.reset_index(drop=True)) #TargetEncoder
df_prizn.head()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


,work_year,experience_level,salary_currency,employee_residence,remote_ratio,company_location,company_size,employment_type_CT,employment_type_FL,employment_type_FT,...,job_title_Principal Data Engineer,job_title_Principal Data Scientist,job_title_Principal Machine Learning Engineer,job_title_Product Data Analyst,job_title_Product Data Scientist,job_title_Research Engineer,job_title_Research Scientist,job_title_Software Data Engineer,job_title_Staff Data Analyst,job_title_Staff Data Scientist
0,1,3,62281.733082,60654.006652,100,57942.497114,0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,2,149366.906638,152822.011651,100,151822.009539,2,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,2,149366.906638,152822.011651,100,151822.009539,2,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,3,149366.906638,132230.933430,100,131924.639077,1,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,3,149366.906638,132230.933430,100,131924.639077,1,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [85]:
from sklearn.preprocessing import MinMaxScaler
normalizer = MinMaxScaler()
df_prizn_norm=df_prizn[chet_col]
df_real_norm_np = normalizer.fit_transform(df_prizn_norm) #Я подумала, что стоит отмасштабировать признаки target encoder, потому что они были огромными
df_norm = pd.DataFrame(data=df_real_norm_np, columns=chet_col)
df_prizn.drop(chet_col,axis=1,inplace=True)
df_prizn=pd.concat([df_prizn, df_norm], axis=1)
df_prizn.head()

,work_year,experience_level,remote_ratio,company_size,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT,job_title_3D Computer Vision Researcher,job_title_AI Developer,...,job_title_Product Data Analyst,job_title_Product Data Scientist,job_title_Research Engineer,job_title_Research Scientist,job_title_Software Data Engineer,job_title_Staff Data Analyst,job_title_Staff Data Scientist,salary_currency,employee_residence,company_location
0,1,3,100,0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.234566,0.172640,0.204778
1,1,2,100,2,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.826902,0.840606,0.961788
2,1,2,100,2,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.826902,0.840606,0.961788
3,1,3,100,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.826902,0.691377,0.801343
4,1,3,100,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.826902,0.691377,0.801343


In [86]:
df_new=pd.concat([df_prizn, df_target], axis=1)
train_big, test=train_test_split(df_new, test_size=0.1, random_state=42) #разделили на тестовую и тренировочную (в тренировочную включили валидационную)
val_dolya=0.1/0.9 #Определили долю валидационной выборки, учитывая то, что она включена в общую тренировочную
train, val=train_test_split(train_big, test_size=val_dolya, random_state=42)#Разделили на валидационную и тренировочную
X_train, y_train=train.drop('salary_in_usd', axis=1), train['salary_in_usd'] #Разделили на признаки и целевую переменную для каждой выборки
X_val, y_val=val.drop('salary_in_usd', axis=1), val['salary_in_usd']
X_test, y_test=test.drop('salary_in_usd', axis=1), test['salary_in_usd']

In [87]:
model=LinearRegression()
model.fit(X_train, y_train) #Обучили на тренировочной
y_pred_val=model.predict(X_val) #Предсказали
mape_val=mean_absolute_percentage_error(y_val, y_pred_val) #Нашли mape
rmse_val=mean_squared_error(y_val, y_pred_val)**0.5 #Нашли rmse
print('MAPE на валидационной выборке:', mape_val)
print('RMSE на валидационной выборке:', rmse_val)
y_pred_test=model.predict(X_test) #То же самое, но для тестовой
mape_test=mean_absolute_percentage_error(y_test, y_pred_test)
rmse_test=mean_squared_error(y_test, y_pred_test)**0.5
print('MAPE на тестовой выборке:', mape_test)
print('RMSE на тестовой выборке:', rmse_test)

MAPE на валидационной выборке: 175261620.9518436
RMSE на валидационной выборке: 119184199537691.28
MAPE на тестовой выборке: 1177114917.516022
RMSE на тестовой выборке: 118680753041267.03


In [88]:
model2=CatBoostRegressor()
model2.fit(X_train, y_train) #Обучили на тренировочной
y_pred_val=model2.predict(X_val) #Предсказали
mape_val=mean_absolute_percentage_error(y_val, y_pred_val) #Нашли mape
rmse_val=mean_squared_error(y_val, y_pred_val)**0.5 #Нашли rmse
print('MAPE на валидационной выборке:', mape_val)
print('RMSE на валидационной выборке:', rmse_val)
y_pred_test=model2.predict(X_test) #То же самое, но для тестовой
mape_test=mean_absolute_percentage_error(y_test, y_pred_test)
rmse_test=mean_squared_error(y_test, y_pred_test)**0.5
print('MAPE на тестовой выборке:', mape_test)
print('RMSE на тестовой выборке:', rmse_test)

Learning rate set to 0.048712
0:	learn: 61324.3310104	total: 4.81ms	remaining: 4.8s
1:	learn: 60394.9170470	total: 15.7ms	remaining: 7.81s
2:	learn: 59562.7938747	total: 19.6ms	remaining: 6.5s
3:	learn: 58715.9586714	total: 23ms	remaining: 5.72s
4:	learn: 58084.3111304	total: 32.5ms	remaining: 6.46s
5:	learn: 57384.7352898	total: 37.5ms	remaining: 6.2s
6:	learn: 56688.4755903	total: 40.7ms	remaining: 5.77s
7:	learn: 56142.4610794	total: 50.2ms	remaining: 6.22s
8:	learn: 55657.1655780	total: 54.6ms	remaining: 6.01s
9:	learn: 55139.4490745	total: 57ms	remaining: 5.64s
10:	learn: 54617.6130258	total: 66.4ms	remaining: 5.97s
11:	learn: 54145.8865490	total: 69.5ms	remaining: 5.72s
12:	learn: 53736.5341530	total: 77.6ms	remaining: 5.89s
13:	learn: 53335.9380585	total: 79.6ms	remaining: 5.61s
14:	learn: 52968.0346409	total: 83.1ms	remaining: 5.46s
15:	learn: 52633.7981835	total: 90.5ms	remaining: 5.57s
16:	learn: 52268.5443003	total: 92.7ms	remaining: 5.36s
17:	learn: 51980.4658717	total: 96.

Можно сделать вывод, что модель catboost работает намного лучше линейной модели, поскольку ее показатели mape и rmse намного меньше. Значит, выберем модель catboost для дальнейших заданий

## Задание 5 (0.5 балла)

Покажите, где ошибается ваша модель. Выведите топ 20 примеров с наибольшей ошибкой. Проанализируете их, постарайтесь выделить группы с причинами ошибок в моделе. Какие выводы можно сделать? Что стоит изменить в данных чтобы улучшить качество модели?

In [89]:
import numpy as np
X_train=X_train.reset_index(drop=True)
y_train=y_train.reset_index(drop=True)
y_pred_train=model2.predict(X_train) #Посмотрим на тестовой выборке, где ошибается наша модель
error=np.abs(y_train-y_pred_train) #Модуль ошибки
sorted_indices=np.argsort(error)[::-1] #Отсортировали
N=20
er_df=pd.DataFrame(columns=df_prizn.columns) #В этот ДФ запишем все наши ошибки
for i in range(N):
    index = sorted_indices[i]
    er_df=pd.concat([er_df, X_train.iloc[[index]]], axis=0)
er_df

,work_year,experience_level,remote_ratio,company_size,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT,job_title_3D Computer Vision Researcher,job_title_AI Developer,...,job_title_Product Data Analyst,job_title_Product Data Scientist,job_title_Research Engineer,job_title_Research Scientist,job_title_Software Data Engineer,job_title_Staff Data Analyst,job_title_Staff Data Scientist,salary_currency,employee_residence,company_location
1373,2,3,100,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.826902,0.840606,0.961788
2141,3,3,50,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.234566,0.642712,0.764640
2496,2,2,0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.826902,0.840606,0.961788
2403,2,3,0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.826902,0.840606,0.961788
1991,3,2,100,0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.618357,0.600916,0.734736
1146,1,3,100,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.826902,0.840606,0.961788
2136,2,2,0,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.381271,0.363492,0.438201
1626,3,2,50,2,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.234566,0.256927,0.384811
1089,4,0,0,2,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.234566,0.437684,0.460049
2120,1,3,100,1,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.826902,0.840606,0.961788


После анализа ошибок я заметила:


1.   remote_ratio очень большой (100,50), хотя у нас все значения остальные максимум до 4. Стоит отмасштабировать этот признак
2.   У всех ошибок employment_type_FT равен 1 (то есть employment_type имеет значение FT). Может нормировку данных проводить по группам (Full-Time отдельно и Contract отдельно)
3. 0.826902	- это самое частое значение у salary_currency, может здесь нужен другой тип кодировки



## Задание 6 (0.5 балла)

Придумайте признаки для улучшения качества модели на основе предыдущего пункта. Как вам кажется какими признаками можно улучшить качетсво модели? Реализуйте признаки и проверьте улучшилось ли качество модели. Полный бал за задание ставится даже при отсутствии улучшения качества модели, важно попробовать проверить свои гипотезы и сделать выводы

## Задание 7 (0.5 балла)

Теперь используете исключительно catboost. Обучить модель. Подберите оптимальные гиперпараметры. Используйте pool для передачи данных в модель с указанием какие признаки категориальные, а какие нет с помощью параметры cat_features. Оцените качество итоговой модели, скорость обучения и скорость предсказания

In [90]:
model = CatBoostRegressor()
from catboost import Pool
cat=X_train.columns.to_list()
cat=[cat[1]]+[cat[3]] #Взяли нормально закодированные категориальные признаки
cat_features=cat
train_pool=Pool(X_train, y_train, cat_features=cat_features)
model.fit(train_pool) #Обучили на тренировочной
y_pred_val=model.predict(X_val) #Предсказали
mape_val=mean_absolute_percentage_error(y_val, y_pred_val) #Нашли mape
rmse_val=mean_squared_error(y_val, y_pred_val)**0.5 #Нашли rmse
print('MAPE1 на валидационной выборке:', mape_val)
print('RMSE1 на валидационной выборке:', rmse_val)

model2=CatBoostRegressor(learning_rate=0.07, iterations=150, random_seed=0, train_dir="learing_rate_0.07") #Попробуем добавить гиперпараметры
model2.fit(X_train,y_train) #Обучили на тренировочной
y_pred_val=model2.predict(X_val) #Предсказали
mape_val=mean_absolute_percentage_error(y_val, y_pred_val) #Нашли mape
rmse_val=mean_squared_error(y_val, y_pred_val)**0.5 #Нашли rmse
print('MAPE2 на валидационной выборке:', mape_val)
print('RMSE2 на валидационной выборке:', rmse_val)

model3=CatBoostRegressor(learning_rate=0.1, iterations=100, random_seed=0, train_dir="learing_rate_0.1") #Попробуем их поменять
model3.fit(X_train,y_train) #Обучили на тренировочной
y_pred_val=model3.predict(X_val) #Предсказали
mape_val=mean_absolute_percentage_error(y_val, y_pred_val) #Нашли mape
rmse_val=mean_squared_error(y_val, y_pred_val)**0.5 #Нашли rmse
print('MAPE3 на валидационной выборке:', mape_val)
print('RMSE3 на валидационной выборке:', rmse_val)

model4=CatBoostRegressor(learning_rate=0.1, iterations=100, random_seed=0, train_dir="learing_rate_0.1") #Попробуем их оставить и добавить Pool сюда
model4.fit(train_pool) #Обучили на тренировочной
y_pred_val=model4.predict(X_val) #Предсказали
mape_val=mean_absolute_percentage_error(y_val, y_pred_val) #Нашли mape
rmse_val=mean_squared_error(y_val, y_pred_val)**0.5 #Нашли rmse
print('MAPE4 на валидационной выборке:', mape_val)
print('RMSE4 на валидационной выборке:', rmse_val)


Learning rate set to 0.048712
0:	learn: 61311.1032059	total: 4.36ms	remaining: 4.36s
1:	learn: 60272.6035944	total: 7.98ms	remaining: 3.98s
2:	learn: 59366.8863894	total: 11.5ms	remaining: 3.81s
3:	learn: 58520.1119820	total: 15ms	remaining: 3.73s
4:	learn: 57668.9677067	total: 18.1ms	remaining: 3.59s
5:	learn: 56929.4928256	total: 21.7ms	remaining: 3.59s
6:	learn: 56325.1276536	total: 25.3ms	remaining: 3.59s
7:	learn: 55707.9808521	total: 28.7ms	remaining: 3.56s
8:	learn: 55180.0451767	total: 32ms	remaining: 3.52s
9:	learn: 54633.4252888	total: 35.4ms	remaining: 3.5s
10:	learn: 54121.3730483	total: 39.3ms	remaining: 3.54s
11:	learn: 53669.4059462	total: 42.7ms	remaining: 3.51s
12:	learn: 53282.7360586	total: 45.9ms	remaining: 3.48s
13:	learn: 52839.2233491	total: 49.2ms	remaining: 3.46s
14:	learn: 52546.1844767	total: 50.7ms	remaining: 3.33s
15:	learn: 52157.6572807	total: 54ms	remaining: 3.32s
16:	learn: 51842.0927691	total: 57.8ms	remaining: 3.34s
17:	learn: 51542.4278873	total: 61.

In [91]:
#Возьмем в качестве итога последнюю, оценим ее качество и время
y_pred_test=model4.predict(X_test) #Предсказали
mape_test=mean_absolute_percentage_error(y_test, y_pred_test) #Нашли mape
rmse_test=mean_squared_error(y_test, y_pred_test)**0.5 #Нашли rmse
print('MAPE4 на тестовой выборке:', mape_test)
print('RMSE4 на тестовой выборке:', rmse_test)
import time
start_time = time.time()
model4.fit(train_pool)
end_time = time.time()
print('Время обучения:', end_time - start_time) #Время обучения
start_time=time.time()
y_pred=model4.predict(X_test)
end_time=time.time()
print('Время предсказания:', end_time - start_time) #Время предсказания

MAPE4 на тестовой выборке: 0.4038812822392581
RMSE4 на тестовой выборке: 50390.42592831037
0:	learn: 60481.6707059	total: 2.21ms	remaining: 218ms
1:	learn: 58886.7599426	total: 4.75ms	remaining: 233ms
2:	learn: 57535.7991821	total: 9.12ms	remaining: 295ms
3:	learn: 56372.8477210	total: 11.3ms	remaining: 271ms
4:	learn: 55451.7562305	total: 14ms	remaining: 266ms
5:	learn: 54552.1959539	total: 16.2ms	remaining: 254ms
6:	learn: 53973.4377131	total: 18.4ms	remaining: 244ms
7:	learn: 53312.2262670	total: 20.7ms	remaining: 238ms
8:	learn: 52869.8423546	total: 22.5ms	remaining: 227ms
9:	learn: 52454.7247058	total: 24.7ms	remaining: 223ms
10:	learn: 51833.3599453	total: 26.9ms	remaining: 217ms
11:	learn: 51235.6956155	total: 29.4ms	remaining: 216ms
12:	learn: 50790.5996159	total: 30.8ms	remaining: 206ms
13:	learn: 50366.3384061	total: 32.9ms	remaining: 202ms
14:	learn: 50019.3390097	total: 34.8ms	remaining: 197ms
15:	learn: 49630.5414368	total: 36.8ms	remaining: 193ms
16:	learn: 49318.7600072	

In [92]:
#Теперь подберем гиперпараметры перебором GridSearchCV
from sklearn.model_selection import GridSearchCV
params = {
    'iterations': [100,150],
    'learning_rate': [0.1, 0.01, 0.001],
    'max_depth': [3,5,7], #Список параметров и их значений
}
model=catboost.CatBoostRegressor()
grid_search=GridSearchCV(model, params) #Подбираем
grid_search.fit(X_train, y_train) #Обучаем
print("Лучшие параметры:", grid_search.best_params_) #Нашли лучшие, далее их подставим

Выходные данные были обрезаны до нескольких последних строк (5000).
101:	learn: 44068.8912049	total: 185ms	remaining: 87ms
102:	learn: 44052.0146031	total: 187ms	remaining: 85.1ms
103:	learn: 44041.2528988	total: 188ms	remaining: 83.2ms
104:	learn: 44009.8132631	total: 190ms	remaining: 81.3ms
105:	learn: 43998.5697311	total: 191ms	remaining: 79.5ms
106:	learn: 43985.8139712	total: 193ms	remaining: 77.6ms
107:	learn: 43949.7418461	total: 195ms	remaining: 75.7ms
108:	learn: 43939.0342409	total: 196ms	remaining: 73.8ms
109:	learn: 43923.1225953	total: 198ms	remaining: 71.9ms
110:	learn: 43886.3705722	total: 199ms	remaining: 70ms
111:	learn: 43856.4885907	total: 201ms	remaining: 68.2ms
112:	learn: 43809.1257841	total: 202ms	remaining: 66.3ms
113:	learn: 43800.7863572	total: 204ms	remaining: 64.4ms
114:	learn: 43775.1351206	total: 206ms	remaining: 62.6ms
115:	learn: 43749.7373985	total: 207ms	remaining: 60.7ms
116:	learn: 43741.3186421	total: 209ms	remaining: 58.9ms
117:	learn: 43698.570411

In [93]:
model=CatBoostRegressor(iterations=150, learning_rate=0.1, max_depth=5) #Модель с лучшими гиперпараметрами
model.fit(X_train,y_train)
y_pred_test=model.predict(X_test) #Предсказали
mape_test=mean_absolute_percentage_error(y_test, y_pred_test) #Нашли mape
rmse_test=mean_squared_error(y_test, y_pred_test)**0.5 #Нашли rmse
print('MAPE на тестовой выборке:', mape_test)
print('RMSE на тестовой выборке:', rmse_test)
import time
start_time = time.time()
model.fit(X_train,y_train)
end_time = time.time()
print('Время обучения:', end_time - start_time) #Время обучения
start_time=time.time()
y_pred=model.predict(X_test)
end_time=time.time()
print('Время предсказания:', end_time - start_time) #Время предсказания

0:	learn: 60313.4401390	total: 1.15ms	remaining: 171ms
1:	learn: 58557.5857994	total: 2.19ms	remaining: 162ms
2:	learn: 56952.7913620	total: 3.22ms	remaining: 158ms
3:	learn: 55765.1471621	total: 4.35ms	remaining: 159ms
4:	learn: 54745.2313305	total: 5.61ms	remaining: 163ms
5:	learn: 53751.2707993	total: 6.81ms	remaining: 164ms
6:	learn: 53021.6101742	total: 8.04ms	remaining: 164ms
7:	learn: 52454.1968639	total: 9.27ms	remaining: 165ms
8:	learn: 51894.7237312	total: 10.5ms	remaining: 164ms
9:	learn: 51341.9466688	total: 11.6ms	remaining: 163ms
10:	learn: 50941.4827571	total: 12.8ms	remaining: 162ms
11:	learn: 50477.9793394	total: 13.8ms	remaining: 159ms
12:	learn: 50178.5875903	total: 15ms	remaining: 159ms
13:	learn: 49921.1898359	total: 16.3ms	remaining: 158ms
14:	learn: 49558.0331163	total: 17.5ms	remaining: 157ms
15:	learn: 49335.3733917	total: 18.7ms	remaining: 157ms
16:	learn: 49126.5329602	total: 20.5ms	remaining: 160ms
17:	learn: 48924.3981250	total: 21.7ms	remaining: 159ms
18:	

## Задание 8 (0.5 балла)

Теперь используете исключительно xgboost. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

In [94]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
# Здесь уже сразу буду подбирать параметры только 2 способом (перебором GridSearchCV)
params = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [100],
    'gamma': [0.1, 0.2]
}
model=xgb.XGBRegressor()
grid_search=GridSearchCV(model, params)
grid_search.fit(X_train, y_train)
print("Лучшие параметры:", grid_search.best_params_) #Все то же самое, просто для другой модели

Лучшие параметры: {'gamma': 0.1, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}


In [95]:
model=xgb.XGBRegressor(gamma=0.1, learning_rate=0.1, max_depth=5,n_estimators=100) #Модель с лучшими параметрами
model.fit(X_train,y_train)
y_pred_test=model.predict(X_test) #Предсказали
mape_test=mean_absolute_percentage_error(y_test, y_pred_test) #Нашли mape
rmse_test=mean_squared_error(y_test, y_pred_test)**0.5 #Нашли rmse
print('MAPE на тестовой выборке:', mape_test)
print('RMSE на тестовой выборке:', rmse_test)
import time
start_time = time.time()
model.fit(X_train,y_train)
end_time = time.time()
print('Время обучения:', end_time - start_time) #Время обучения
start_time=time.time()
y_pred=model.predict(X_test)
end_time=time.time()
print('Время предсказания:', end_time - start_time) #Время предсказания

MAPE на тестовой выборке: 0.3869692538674379
RMSE на тестовой выборке: 49857.825372512016
Время обучения: 1.1039319038391113
Время предсказания: 0.0070002079010009766


## Задание 9 (0.5 балла)

Теперь используете исключительно lightgbm. Обучить модель. Подберите оптимальные гиперпараметры. Закодируйте категориальные переменные, как хотите. Оцените качество итоговой модели, скорость обучения и скорость предсказания.

In [96]:
import lightgbm as lgb
# Определение параметров для подбора
params = {
    'max_depth': [3, 5],
    'learning_rate': [0.1, 0.01],
    'n_estimators': [50, 100],
    'min_child_samples': [20,40],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.6, 1.0]
}
model=lgb.LGBMRegressor()
grid_search=GridSearchCV(model, params)
grid_search.fit(X_train, y_train)
print("Лучшие параметры:", grid_search.best_params_)

Лучшие параметры: {'colsample_bytree': 0.6, 'learning_rate': 0.1, 'max_depth': 5, 'min_child_samples': 20, 'n_estimators': 50, 'subsample': 0.8}


In [97]:
model=lgb.LGBMRegressor(colsample_bytree= 0.6, learning_rate=0.1, max_depth=5, min_child_samples= 20, n_estimators=50, subsample=0.8)
model.fit(X_train,y_train)
y_pred_test=model.predict(X_test) #Предсказали
mape_test=mean_absolute_percentage_error(y_test, y_pred_test) #Нашли mape
rmse_test=mean_squared_error(y_test, y_pred_test)**0.5 #Нашли rmse
print('MAPE на тестовой выборке:', mape_test)
print('RMSE на тестовой выборке:', rmse_test)
import time
start_time = time.time()
model.fit(X_train,y_train)
end_time = time.time()
print('Время обучения:', end_time - start_time) #Время обучения
start_time=time.time()
y_pred=model.predict(X_test)
end_time=time.time()
print('Время предсказания:', end_time - start_time) #Время предсказания

MAPE на тестовой выборке: 0.4053623091333894
RMSE на тестовой выборке: 50629.12246268273
Время обучения: 0.0320584774017334
Время предсказания: 0.0034093856811523438


## Задание 10 (0.5 балла)

Сделайте выводы про модели бустинга, какая из моделей показала лучший результат по качеству, скорости обучения и скорости предсказания

Выводы:


1.   Лучшее Mape на тестовой выборке: модель XGBRegressor
2.   Лучшее RMSE на тестовой выборке: модель CatBoostRegressor
3. Самое быстрое время обучения: модель LGBMRegressor
4. Самое быстрое время предсказания: модель LGBMRegressor

 По итогу можно сделать вывод, что лучшее качество показывают модели  XGBRegressor и CatBoostRegressor, однако LGBMRegressor хорош в том, что быстро обучается и быстро выдает предсказания (из-за этого намного быстрее работал и перебор гиперпараметров)


